# CS661 Assignment 2 Group 24
24_241110035_241110089_Assignment2

Khushwant Kaswan - 241110035 - khushwantk24@iitk.ac.in

Senthil Ganesh - 241110089 - senthil24@iitk.ac.in

In case of any error while running the code, please contact any of the group members.

---

Install the required packages (in virtual env):

```bash
python3 -m venv myenv`
source myenv/bin/activate
pip install plotly ipywidgets numpy vtk jupyter
```

Alternatively we have added requirments.txt containing all python packages of our virtual env.
You can also do the following:

```bash
pip install -r requirments.txt
```

Code is running fine in both vscode and Jupyter Labs.

Demo video of both present in the folder as Demo-1 and Demo-2.

---


Jupyter Labs issue: One of the plot or both disappers on first run.
This issue typically occurs due to timing/initialization differences between Jupyter Lab's widget rendering and Plotly's Figure display.We can just move the slider and everything will work fine as expected.


In [ ]:
import numpy as np
import vtk
from vtk.util import numpy_support
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import FloatSlider, Button, HBox, VBox, Output, Label
from IPython.display import display, clear_output


In [ ]:
# Path to VTI file
DATA_FILE = "mixture.vti"

In [ ]:
# Class to handle volume data loading and processing
class VolumeDataHandler:

    def __init__(self, file_path):
        self.file_path = file_path
        self.volume_data = None
        self.coordinates = None
        self.load_data()

    # Load data from VTI file using VTK
    def load_data(self):

        # Create reader for XML image data
        reader = vtk.vtkXMLImageDataReader()
        reader.SetFileName(self.file_path)
        reader.Update()

        # Extract data from reader
        vtk_data = reader.GetOutput()
        scalar_array = vtk_data.GetPointData().GetScalars()
        dimensions = vtk_data.GetDimensions()

        # Get spatial information
        data_origin = vtk_data.GetOrigin()
        data_spacing = vtk_data.GetSpacing()

        # Generate coordinate arrays
        coord_arrays = []
        for i in range(3):
            start = data_origin[i]
            step = data_spacing[i]
            size = dimensions[i]
            coord_arrays.append(np.linspace(start, start + step * (size - 1), size))

        # Create meshgrid for coordinates
        X, Y, Z = np.meshgrid(coord_arrays[0], coord_arrays[1], coord_arrays[2], indexing='ij')
        self.coordinates = (X, Y, Z)

        # Reshape scalar data to match dimensions
        self.volume_data = np.array(scalar_array).reshape(dimensions, order='F')

    def get_data(self):
        # Return coordinates and volume data
        return self.coordinates + (self.volume_data,)


In [ ]:
# Visualization Manager class
class VisualizationManager:
    """Manages the visualization components and user interface"""

    def __init__(self, data_handler):
        # Get data from handler
        self.X, self.Y, self.Z, self.volume_data = data_handler.get_data()

        # Set initial values
        self.initial_isovalue = 0.0
        self.current_isovalue = self.initial_isovalue
        self.filter_range = 0.25  # Range for histogram filtering

        # Create UI components
        self._setup_ui()

    # Set up UI components
    def _setup_ui(self):

        # Create output widgets
        self.isosurface_output = Output()
        self.histogram_output = Output()
        self.status_display = Label(value="Ready")

        # Style the status label
        self.status_display.style = {'font_weight': 'bold'}
        self.status_display.layout = widgets.Layout(justify_content="center")

        # Create slider for isovalue control
        data_min, data_max = self.volume_data.min(), self.volume_data.max()
        print(f"Data range: {data_min} to {data_max}")

        self.isovalue_slider = FloatSlider(
            value=self.initial_isovalue,
            min=data_min,
            max=data_max,
            step=0.01,
            description='Isoval:',
            continuous_update=False,
            layout=widgets.Layout(width="40%")
        )

        # Create reset button
        self.reset_button = Button(description="Reset")

        # Set up event handlers
        self.isovalue_slider.observe(self._on_slider_change, 'value')
        self.reset_button.on_click(self._on_reset_clicked)

        # Arrange UI layout
        self.top_controls = HBox(
            [self.isovalue_slider, self.reset_button],
            layout=widgets.Layout(width="100%")
        )

        self.status_row = HBox(
            [self.status_display],
            layout=widgets.Layout(width="100%", justify_content="center")
        )

        self.visualization_row = HBox(
            [self.isosurface_output, self.histogram_output],
            layout=widgets.Layout(width="100%")
        )

        # Set individual widget of layouts
        self.isosurface_output.layout = widgets.Layout(width="50%")
        self.histogram_output.layout = widgets.Layout(width="50%")

    def _on_slider_change(self, change):
        # Handle slider value changes
        self.current_isovalue = change['new']
        self.update_visualizations()

    def _on_reset_clicked(self, button):
        # Handle reset button clicks
        # Temporarily removing observer to prevent double execution
        self.isovalue_slider.unobserve(self._on_slider_change, 'value')

        # Reset values
        self.current_isovalue = self.initial_isovalue
        self.isovalue_slider.value = self.initial_isovalue

        # Re-attach observer
        self.isovalue_slider.observe(self._on_slider_change, 'value')

        # Update visualizations with reset state
        self.update_visualizations(is_reset=True,show_full=True)

    def update_visualizations(self, is_reset=False,show_full=False):
        # Update both visualizations based on current state
        # Update status message
        if is_reset:
            self.status_display.value = "Resetting visualization..."
        else:
            self.status_display.value = "Updating visualization..."

        # Update isosurface plot
        self._update_isosurface()

        # Update histogram plot
        self._update_histogram(show_full=show_full)

        # Update status when complete
        self.status_display.value = "Visualization updated successfully"

    def _update_isosurface(self):
        # Update the isosurface visualization
        with self.isosurface_output:
            clear_output(wait=True)

            # Create isosurface figure
            iso_fig = go.Figure(
                data=[go.Isosurface(
                    x=self.X.flatten(),
                    y=self.Y.flatten(),
                    z=self.Z.flatten(),
                    value=self.volume_data.flatten(),
                    isomin=self.current_isovalue,
                    isomax=self.current_isovalue,
                    cmin=self.volume_data.min(),
                    cmax=self.volume_data.max(),
                    surface_count=1,
                    showscale=False,
                    colorscale="plasma",
                    caps=dict(x_show=False, y_show=False, z_show=False),
                    hovertemplate="x: %{x}<br>y: %{y}<br>z: %{z}<br>value: " + str(self.current_isovalue) + "<extra></extra>",
                )]
            )

            # Configure layout
            iso_fig.update_layout(
                # title="3D Isosurface View",
                width=600,
                height=600,
                scene=dict(
                    xaxis=dict(showticklabels=False),
                    yaxis=dict(showticklabels=False),
                    zaxis=dict(showticklabels=False),
                )
            )

            display(iso_fig)

    def _update_histogram(self, show_full=False):
        # Update the histogram visualization
        with self.histogram_output:
            clear_output(wait=True)

            # Determine histogram data and title
            if show_full:
                # Use all data for histogram
                hist_data = self.volume_data.flatten()
                title = "Histogram of Entire Dataset"
            else:
                # Filter data around current isovalue
                lower_bound = self.current_isovalue - self.filter_range
                upper_bound = self.current_isovalue + self.filter_range

                # Use boolean mask for filtering
                mask = (self.volume_data >= lower_bound) & (self.volume_data <= upper_bound)
                hist_data = self.volume_data[mask].flatten()

                title = f"Histogram for values [{lower_bound:.2f} to {upper_bound:.2f}]"

            # Create histogram figure
            hist_fig = go.Figure(
                data=[go.Histogram(
                    x=hist_data,
                    marker_color='blue',
                    nbinsx=30
                )]
            )


            # Configure layout
            hist_fig.update_layout(
                title=title,
                width=600,
                height=600,
                xaxis_title='Vortex Scalar Value',
                yaxis_title='Frequency',
                # bargap=0.05
            )

            display(hist_fig)

    def display(self):
        # Display the complete UI
        display(self.top_controls, self.status_row, self.visualization_row)

        # Initial visualization
        self.update_visualizations(is_reset=False,show_full=True)


In [ ]:
# Main execution
def main():
    # Load data
    data_handler = VolumeDataHandler(DATA_FILE)

    # Create and display visualization
    viz_manager = VisualizationManager(data_handler)
    viz_manager.display()

# Run the application
main()


Data range: -0.9935540556907654 to 0.43280163407325745
